# Instruments and Intermediate Measurements Tutorial
This tutorial will demonstrate how perform tomography on models which, in addition to normal gates, contain *quantum instruments*.  Quantum instruments are maps that act on a qubit state (density matrix) and produce a qubit state along with a classical outcome.  That is, instruments are maps from $\mathcal{B}(\mathcal{H})$, the space of density matrices, to $\mathcal{B}(\mathcal{H}) \otimes K(n)$, where $K(n)$ is a classical space of $n$ elements.

In pyGSTi, instruments are represented as collections of gates, one for each classical "outcome" of the instrument.  This tutorial will demonstrate how to add instruments to `Model` objects, compute probabilities using such `Model`s, and ultimately perform tomography on them.  We'll start with a few familiar imports:

In [12]:
import pygsti
from pygsti.modelpacks import smq1Q_XYI as std
import numpy as np
from pygsti.modelmembers.instruments import Instrument
from pprint import pprint

## Instrument construction
Next, we'll add an instrument to our "standard" model - a 1-qubit model containing $I$, $X(\pi/2)$, and $Y(\pi/2)$ gates.  The ideal instrument will be named `"Iz"` (all instrument names must begin with `"I"`), and consist of perfect projectors onto the 0 and 1 states.  Instead of labelling the associated outcomes "0" and "1", which might me most logical, we'll name them "p0" and "p1" so it's easier to distinguish them from the final POVM outcomes which *are* labelled "0" and "1".

In [13]:
#Make a copy so we don't modify the original
mdl_ideal = std.target_model()

# Create and add the ideal instrument
E0 = mdl_ideal.effects['0']
E1 = mdl_ideal.effects['1']
Gmz_plus  = np.dot(E0,E0.T) # note effect vectors are stored as column vectors
Gmz_minus = np.dot(E1,E1.T)
inst = Instrument({'p0': Gmz_plus, 'p1': Gmz_minus})
mdl_ideal[('Iz',0)] = inst

In order to generate some simulated data later on, we'll now create a noisy version of `mdl_ideal` by depolarizing the state preparation, gates, and POVM, and also rotating the basis that is measured by the instrument and POVM.

In [14]:
mdl_noisy = mdl_ideal.depolarize(op_noise=0.005, spam_noise=0.01)
mdl_noisy = mdl_noisy.rotate(max_rotate=0.025, seed=2048)
mdl_noisy.effects.depolarize(0.01)
mdl_ideal.convert_members_inplace('CPTPLND')
mdl_noisy.convert_members_inplace('CPTPLND')

## Generating probabilities 
Instrument labels (e.g. `"Iz"`) may be included within `Circuit` objects, and `Model` objects are able to compute probabilities for them just like normal (non-instrument) operation sequences.  The difference is that probabilities are labeled by tuples of instrument and POVM outcomes - referred to as **"outcome tuples"** - one for each instrument and one for the final POVM:

In [15]:
for mdl in [mdl_ideal, mdl_noisy]:
    pprint(dict(mdl.probabilities( pygsti.circuits.Circuit(( ('Gxpi2',0) , ('Iz',0) )) )))
    print()

{('p0', '0'): 0.5000000000000003,
 ('p0', '1'): 1.4597476004180728e-17,
 ('p1', '0'): 5.073268178784763e-18,
 ('p1', '1'): 0.5}

{('p0', '0'): 0.481367923790963,
 ('p0', '1'): 0.0024189342904068554,
 ('p1', '0'): 0.0025810657095932145,
 ('p1', '1'): 0.5136320762090365}



In [16]:
for mdl in [mdl_ideal, mdl_noisy]:
    pprint(dict(mdl.probabilities( pygsti.circuits.Circuit(( ('Iz',0), ('Gxpi2',0) , ('Iz',0) )) )))
    print()

{('p0', 'p0', '0'): 0.5000000000000002,
 ('p0', 'p0', '1'): -1.6613637599827203e-18,
 ('p0', 'p1', '0'): -1.1185571585378685e-17,
 ('p0', 'p1', '1'): 0.4999999999999999,
 ('p1', 'p0', '0'): 1.625883976416346e-17,
 ('p1', 'p0', '1'): 1.8939874217871704e-34,
 ('p1', 'p1', '0'): 4.5374861265545935e-34,
 ('p1', 'p1', '1'): 1.625883976416347e-17}

{('p0', 'p0', '0'): 0.4775917799207901,
 ('p0', 'p0', '1'): 0.002399958693069308,
 ('p0', 'p1', '0'): 0.0025624981205110728,
 ('p0', 'p1', '1'): 0.5099371259816915,
 ('p1', 'p0', '0'): 0.0038579004920827257,
 ('p1', 'p0', '1'): 1.938643463358172e-05,
 ('p1', 'p1', '0'): 1.8156751786106926e-05,
 ('p1', 'p1', '1'): 0.003613193605435256}



In fact, pyGSTi *always* labels probabilties using outcome tuples, it's just that in the non-instrument case they're always 1-tuples and by `OutcomeLabelDict` magic can be treated as if they were just strings:

In [17]:
probs = mdl_ideal.probabilities( pygsti.circuits.Circuit([('Gxpi2',0)]) )
print("probs = ",dict(probs))
print("probs['0'] = ", probs['0']) #This works...
print("probs[('0',)] = ", probs[('0',)]) # and so does this.

probs =  {('0',): 0.5000000000000004, ('1',): 0.5}
probs['0'] =  0.5000000000000004
probs[('0',)] =  0.5000000000000004


## Performing tomography
Now let's perform tomography on a model that includes instruments. 
First, we'll build an experiment design. This notebook's experiment design makes a minor modification to the default design for our working modelpack (smq1Q_XYI). 

In [18]:
germs = std.germs()
germs += [pygsti.circuits.Circuit([('Iz', 0)])]
ed = std.create_gst_experiment_design(max_max_length=8, germs=germs)

### Simulated data generation
Next, we generate data using `mdl_noisy` in exactly the same way as we would for any other model. We write the data to disk so you can see how datasets look when they include measurement data.

In [19]:
ds = pygsti.data.simulate_data(mdl_noisy, ed.all_circuits_needing_data, 10_000, 'multinomial', seed=2018)
pygsti.io.write_dataset("../../tutorial_files/intermediate_meas_dataset.txt", ds)

Notice the format of [intermediate_meas_dataset.txt](../../tutorial_files/intermediate_meas_dataset.txt), which includes a column for each distinct outcome tuple.  Since not all experiments contain data for all outcome tuples, the `"--"` is used as a placeholder.  Now that the data is generated, we run LGST or LSGST just like we would for any other model:

### Running the GST fit and generating a report

In [20]:
from pygsti.protocols import StandardGST, ProtocolData
gst = StandardGST(
    modes=('full TP', 'CPTPLND'), target_model=mdl_ideal, verbosity=2
)
pd = ProtocolData(ed, ds)
res = gst.run(pd)

-- Std Practice:  Iter 1 of 2  (full TP) --: 
  --- Iterative GST: [##################################################] 100.0%  592 circuits ---
-- Std Practice:  Iter 2 of 2  (CPTPLND) --: 
  --- Iterative GST: [##################################################] 100.0%  592 circuits ---


In [21]:
from pygsti.report import construct_standard_report
report_dir = '../../tutorial_files/cptp-instrument-report'
report_object = construct_standard_report(res, title='CPTP intrument GST')
report_object.write_html(report_dir)

Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI


/Users/rjmurr/Documents/pygsti-general/pyGSTi/pygsti/models/model.py:145: UserWarning:

Model.num_modeltest_params could not obtain number of *non-gauge* parameters - using total instead

/Users/rjmurr/Documents/pygsti-general/pyGSTi/pygsti/tools/optools.py:152: UserWarning:

The input matrix a is not trace-1 up to tolerance 1.4901161193847656e-08. Beware result!

/Users/rjmurr/Documents/pygsti-general/pyGSTi/pygsti/tools/optools.py:154: UserWarning:

The input matrix b is not trace-1 up to tolerance 1.4901161193847656e-08. Beware result!

/Users/rjmurr/Documents/pygsti-general/pyGSTi/pygsti/tools/optools.py:179: UserWarning:


            Input matrix is not PSD up to tolerance 1.4901161193847656e-08.
            We'll project out the bad eigenspaces to only work with the PSD part.
            

/Users/rjmurr/Documents/pygsti-general/pyGSTi/pygsti/report/workspacetables.py:2741: RuntimeWarning:

divide by zero encountered in log

/Users/rjmurr/Documents/pygsti-general/pyGSTi/pygsti/fo

**Thats it!**  You've done tomography on a model with intermediate measurments (instruments).

As a bonus, the code below checks for violation of complete positivity of the instrument operations from the two fits. We see that the CPTP fit has no violation, while the full TP fit has small violations.

In [22]:
for mdl in [res.estimates['full TP'].models['stdgaugeopt'], res.estimates['CPTPLND'].models['stdgaugeopt']]:
    for instlbl, inst in mdl.instruments.items():
        print(instlbl)
        for ioplbl, iop in inst.items():
            violation = max(0, pygsti.tools.sum_of_negative_choi_eigenvalues_gate(iop.to_dense(), 'pp'))
            print(ioplbl + ': ' + str(violation) )
        print()

Iz:0
p0: 0.0009697360237423556
p1: 2.6230206864367674e-05

Iz:0
p0: 0
p1: 0

